In [26]:
import vrep 
import sys
import time 
import numpy as np
from tank import *
from skfuzzy import control as ctrl
import skfuzzy as fuzz

## Agresywne hamowanie

In [37]:
def aggresiveSimulation():
    x_speeds = np.arange(-3, 15, 1)
    x_distances = np.arange(0, 5, 1)


    distance = ctrl.Antecedent(x_distances, 'distance')
    target_velocity = ctrl.Consequent(x_speeds, 'target_velocity')

    distance.automf(3)
    target_velocity.automf(3)

    rules = (
        ctrl.Rule(distance['poor'] , target_velocity['poor']),
        ctrl.Rule(distance['good'] , target_velocity['good'])
    )

    driver = ctrl.ControlSystemSimulation(ctrl.ControlSystem(rules=rules))

    vrep.simxFinish(-1) # closes all opened connections, in case any prevoius wasnt finished
    clientID=vrep.simxStart('127.0.0.1',19999,True,True,5000,5) # start a connection

    if clientID!=-1:
        pass
        # print ("Connected to remote API server")
    else:
        # print("Not connected to remote API server")
        sys.exit("Could not connect")

    tank=Tank(clientID)

    err_code,ps_handle = vrep.simxGetObjectHandle(clientID,"Proximity_sensor", vrep.simx_opmode_blocking)

    while True:
        err_code,_,detectedPoint,_,_=vrep.simxReadProximitySensor(clientID,ps_handle,vrep.simx_opmode_streaming)
        dst = np.linalg.norm(detectedPoint)
        c_vel = tank.readVelocity()

        driver.input['distance'] = dst

        driver.compute()

        target = driver.output['target_velocity']
        if target <= 0.5:
            target = 0

        tank.forward(target)
        print(f"\rdst={dst:.2f}, curr_velocity={c_vel:.2f}, target_velocity={target:.2f}", end='')

In [38]:
aggresiveSimulation()

dst=0.00, curr_velocity=0.00, target_velocity=0.0071

KeyboardInterrupt: 

In [61]:
def gradualSimulation():
    x_speeds = np.arange(-5, 13, 0.1)
    x_distances = np.arange(0, 10, 0.1)

    distance = ctrl.Antecedent(x_distances, 'distance')
    curr_velocity = ctrl.Antecedent(x_speeds, 'curr_velocity')
    target_velocity = ctrl.Consequent(x_speeds, 'target_velocity')

    distance.automf(3)
    target_velocity.automf(3)
    curr_velocity.automf(3)

    rules = (
        ctrl.Rule(distance['poor'] & curr_velocity['poor'], target_velocity['poor']),
        ctrl.Rule(distance['poor'] & curr_velocity['average'], target_velocity['poor']),
        ctrl.Rule(distance['poor'] & curr_velocity['good'], target_velocity['average']),
        ctrl.Rule(distance['average'] & curr_velocity['poor'], target_velocity['good']),
        ctrl.Rule(distance['average'] & curr_velocity['average'], target_velocity['good']),
        ctrl.Rule(distance['average'] & curr_velocity['good'], target_velocity['good']),
        ctrl.Rule(distance['good'] & curr_velocity['poor'], target_velocity['good']),
        ctrl.Rule(distance['good'] & curr_velocity['average'], target_velocity['good']),
        ctrl.Rule(distance['good'] & curr_velocity['good'], target_velocity['good']),
    )

    driver = ctrl.ControlSystemSimulation(ctrl.ControlSystem(rules=rules))

    vrep.simxFinish(-1) # closes all opened connections, in case any prevoius wasnt finished
    clientID=vrep.simxStart('127.0.0.1',19999,True,True,5000,5) # start a connection

    if clientID!=-1:
        print ("Connected to remote API server")
    else:
        print("Not connected to remote API server")
        sys.exit("Could not connect")

    tank=Tank(clientID)

    err_code,ps_handle = vrep.simxGetObjectHandle(clientID,"Proximity_sensor", vrep.simx_opmode_blocking)


    while True:
        err_code,_,detectedPoint,_,_=vrep.simxReadProximitySensor(clientID,ps_handle,vrep.simx_opmode_streaming)
        dst = np.linalg.norm(detectedPoint)
        c_vel = tank.readVelocity()

        driver.input['distance'] = dst
        driver.input['curr_velocity'] = c_vel

        driver.compute()

        target = driver.output['target_velocity']
        if target <= 0.5:
            target = 0

        tank.forward(target)
        print(f"\rdst={dst:.2f}, curr_velocity={c_vel:.2f}, target_velocity={target:.2f}", end='')

In [62]:
gradualSimulation()

Connected to remote API server
dst=0.00, curr_velocity=0.00, target_velocity=0.00

KeyboardInterrupt: 